In [ ]:
# Orignial Notebook: https://github.com/MITESHPUTHRANNEU/Speech-Emotion-Analyzer/blob/master/final_results_gender_test.ipynb
# This notebook author: Reza Chu
# Last Editing Date: 31st May 2019

## Python
import os
import random
import sys

import IPython
from IPython.display import Audio
from IPython.display import Image
import matplotlib.pyplot as plt

## Package
import glob 
import keras
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as py
import plotly.tools as tls
import seaborn as sns
import scipy.io.wavfile
import tensorflow as tf
py.init_notebook_mode(connected=True)

## Keras
from keras import regularizers
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras.callbacks import  History, ReduceLROnPlateau, CSVLogger
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical

## Sklearn
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

## Rest
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
from tqdm import tqdm

input_duration=3
# % pylab inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## count no csv for landmark


In [ ]:
landmark_folder='/content/drive/MyDrive/'
landmark_folder = os.path.join(landmark_folder, "landmarks/")
print(landmark_folder)
count=0
emo=np.zeros((6,1))
v=np.zeros((2,1))
dir_list = os.listdir(landmark_folder)
print(dir_list)
for f in dir_list:
        print(f)
        nm = f.split('.')[0].split('-')
        #print(nm)
        #path = '/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/'+ i + '/' +f+ '/frames/' 
        #print(len(nm))
        s= int(nm[1])
        if s==1:
          v[0]+=1
        elif s==2:
          v[1]+=1
        if s==2:
          continue
        actor = int(nm[-2])
        emotion = int(nm[2])-1
        if(emotion==7 or emotion==6):
          continue
        emo[emotion]+=1
        count+=1

/content/drive/MyDrive/landmarks/
['01-01-08-02-01-01-14.csv', '01-02-02-01-01-01-15.csv', '01-02-06-01-02-01-03.csv', '01-01-04-01-02-02-12.csv', '01-02-02-01-01-01-20.csv', '01-01-01-01-01-02-19.csv', '01-01-01-01-02-01-18.csv', '01-02-04-01-02-01-17.csv', '01-01-04-02-02-01-20.csv', '01-01-04-01-02-02-07.csv', '01-01-04-02-02-01-03.csv', '01-02-06-01-02-02-14.csv', '01-01-06-01-02-02-14.csv', '01-01-08-02-02-01-04.csv', '01-01-08-01-02-02-03.csv', '01-01-06-02-02-02-01.csv', '01-01-05-02-02-02-07.csv', '01-01-07-01-01-01-07.csv', '01-02-06-02-02-02-08.csv', '01-01-05-01-02-01-13.csv', '01-02-03-01-01-02-08.csv', '01-02-02-02-01-01-05.csv', '01-02-03-02-01-01-02.csv', '01-01-04-01-01-01-19.csv', '01-01-02-01-02-02-23.csv', '01-01-06-01-01-01-19.csv', '01-02-05-01-01-02-07.csv', '01-02-04-02-01-02-10.csv', '01-01-04-02-01-02-12.csv', '01-02-04-01-02-02-05.csv', '01-02-04-01-01-01-14.csv', '01-02-02-02-02-02-15.csv', '01-01-02-01-02-01-13.csv', '01-02-05-01-01-01-21.csv', '01-01-06-01-

In [ ]:
v

array([[1440.],
       [1012.]])

In [ ]:
emo

array([[ 96.],
       [192.],
       [192.],
       [192.],
       [192.],
       [192.]])

## check video with audio is same video without audio 

In [ ]:
## check videos with and without audio
path3 = '/content/drive/My Drive/Video_Speech'
lav=[]
lvo=[]
cav=0
cvo=0
count=0
video=[]
for subdir, dirs, files in os.walk(path3):
  print(files)
  for file in files:
    if file is not None:
        print(file)
        nm = file.split('.')[0].split('-')
        s= int(nm[0])
        if s==1:
          lav.append(nm[1:])
          cav+=1
          video.append(file)
        elif s==2:
          lvo.append(nm[1:])
          cvo+=1
        count+=1


[]
['01-01-01-01-01-01-07.mp4', '01-01-01-01-02-01-07.mp4', '01-01-01-01-01-02-07.mp4', '01-01-01-01-02-02-07.mp4', '01-01-02-01-02-02-07.mp4', '01-01-02-01-02-01-07.mp4', '01-01-02-01-01-02-07.mp4', '01-01-02-01-01-01-07.mp4', '01-01-02-02-01-01-07.mp4', '01-01-02-02-01-02-07.mp4', '01-01-02-02-02-01-07.mp4', '01-01-02-02-02-02-07.mp4', '01-01-03-01-01-02-07.mp4', '01-01-03-01-02-01-07.mp4', '01-01-03-01-01-01-07.mp4', '01-01-03-02-01-01-07.mp4', '01-01-03-01-02-02-07.mp4', '01-01-03-02-02-01-07.mp4', '01-01-03-02-01-02-07.mp4', '01-01-04-01-01-01-07.mp4', '01-01-03-02-02-02-07.mp4', '01-01-04-01-01-02-07.mp4', '01-01-04-01-02-01-07.mp4', '01-01-04-02-01-01-07.mp4', '01-01-04-02-01-02-07.mp4', '01-01-04-01-02-02-07.mp4', '01-01-04-02-02-02-07.mp4', '01-01-05-01-01-01-07.mp4', '01-01-04-02-02-01-07.mp4', '01-01-05-01-01-02-07.mp4', '01-01-05-02-01-01-07.mp4', '01-01-05-01-02-01-07.mp4', '01-01-05-01-02-02-07.mp4', '01-01-05-02-01-02-07.mp4', '01-01-05-02-02-01-07.mp4', '01-01-05-02-02-

In [ ]:
cav,cvo

(1440, 1440)

In [ ]:
video

['01-01-01-01-01-01-07.mp4',
 '01-01-01-01-02-01-07.mp4',
 '01-01-01-01-01-02-07.mp4',
 '01-01-01-01-02-02-07.mp4',
 '01-01-02-01-02-02-07.mp4',
 '01-01-02-01-02-01-07.mp4',
 '01-01-02-01-01-02-07.mp4',
 '01-01-02-01-01-01-07.mp4',
 '01-01-02-02-01-01-07.mp4',
 '01-01-02-02-01-02-07.mp4',
 '01-01-02-02-02-01-07.mp4',
 '01-01-02-02-02-02-07.mp4',
 '01-01-03-01-01-02-07.mp4',
 '01-01-03-01-02-01-07.mp4',
 '01-01-03-01-01-01-07.mp4',
 '01-01-03-02-01-01-07.mp4',
 '01-01-03-01-02-02-07.mp4',
 '01-01-03-02-02-01-07.mp4',
 '01-01-03-02-01-02-07.mp4',
 '01-01-04-01-01-01-07.mp4',
 '01-01-03-02-02-02-07.mp4',
 '01-01-04-01-01-02-07.mp4',
 '01-01-04-01-02-01-07.mp4',
 '01-01-04-02-01-01-07.mp4',
 '01-01-04-02-01-02-07.mp4',
 '01-01-04-01-02-02-07.mp4',
 '01-01-04-02-02-02-07.mp4',
 '01-01-05-01-01-01-07.mp4',
 '01-01-04-02-02-01-07.mp4',
 '01-01-05-01-01-02-07.mp4',
 '01-01-05-02-01-01-07.mp4',
 '01-01-05-01-02-01-07.mp4',
 '01-01-05-01-02-02-07.mp4',
 '01-01-05-02-01-02-07.mp4',
 '01-01-05-02-

In [ ]:
X_name = 'video_files_name.joblib'
save_dir = '/content/drive/MyDrive/graduation project/video/marwa/new_features/'

savedX = joblib.dump(video, os.path.join(save_dir, X_name))

In [ ]:
new_data = joblib.load('/content/drive/MyDrive/graduation project/video/marwa/new_features/video_files_name.joblib')

In [ ]:
new_data

['01-01-01-01-01-01-07.mp4',
 '01-01-01-01-02-01-07.mp4',
 '01-01-01-01-01-02-07.mp4',
 '01-01-01-01-02-02-07.mp4',
 '01-01-02-01-02-02-07.mp4',
 '01-01-02-01-02-01-07.mp4',
 '01-01-02-01-01-02-07.mp4',
 '01-01-02-01-01-01-07.mp4',
 '01-01-02-02-01-01-07.mp4',
 '01-01-02-02-01-02-07.mp4',
 '01-01-02-02-02-01-07.mp4',
 '01-01-02-02-02-02-07.mp4',
 '01-01-03-01-01-02-07.mp4',
 '01-01-03-01-02-01-07.mp4',
 '01-01-03-01-01-01-07.mp4',
 '01-01-03-02-01-01-07.mp4',
 '01-01-03-01-02-02-07.mp4',
 '01-01-03-02-02-01-07.mp4',
 '01-01-03-02-01-02-07.mp4',
 '01-01-04-01-01-01-07.mp4',
 '01-01-03-02-02-02-07.mp4',
 '01-01-04-01-01-02-07.mp4',
 '01-01-04-01-02-01-07.mp4',
 '01-01-04-02-01-01-07.mp4',
 '01-01-04-02-01-02-07.mp4',
 '01-01-04-01-02-02-07.mp4',
 '01-01-04-02-02-02-07.mp4',
 '01-01-05-01-01-01-07.mp4',
 '01-01-04-02-02-01-07.mp4',
 '01-01-05-01-01-02-07.mp4',
 '01-01-05-02-01-01-07.mp4',
 '01-01-05-01-02-01-07.mp4',
 '01-01-05-01-02-02-07.mp4',
 '01-01-05-02-01-02-07.mp4',
 '01-01-05-02-

In [ ]:
v=new_data[0]
v

'01-01-01-01-01-01-07.mp4'

In [ ]:
v[:-4]

'01-01-01-01-01-01-07'

In [ ]:
def compare(lav,lvo):
  lav.sort()
  lvo.sort()
  if(lav==lvo):
    return "Equal"
  else:
    return "Non equal"

In [ ]:
compare(lav,lvo)

'Equal'

##extract 30 frame from video with using landmark

In [ ]:
import os
import cv2
import numpy as np
from moviepy.editor import VideoFileClip


class VideoProcessor(object):
    def __init__(self, video_path, landmark_path, output_folder, extract_audio):
        # path of the video file
        self.video_path = video_path
        self.landmarks_path = landmark_path
        self.extract_audio = extract_audio
        self.frames_folder = os.path.join(output_folder, "frames")
        self.audios_folder = os.path.join(output_folder, "audios")

        if not os.path.exists(output_folder):
            os.mkdir(output_folder)
        #if not os.path.exists(self.audios_folder):
            #os.mkdir(self.audios_folder)
        if not os.path.exists(self.frames_folder):
            os.mkdir(self.frames_folder)

    def preprocess(self, seq_len=30, target_resolution=(224, 224)):
        """
        extract frames and audio from the video,
        store the cropped frames and audio file in the output folders
        seq_len: how many frames will be extracted from the video.
                  Considering all videos from this dataset have similar duration
                  video_duration = seq_len / fps
        target_resolution: (desired_height, desired_width) of the facial frame extracted
        """
        print("self video path : "+self.video_path)
        video = VideoFileClip(self.video_path, audio=self.extract_audio, target_resolution=target_resolution)
        #if self.extract_audio:
         #   video.audio.write_audiofile(os.path.join(self.audios_folder, "audio.wav"))

        times = list(np.arange(0, video.duration, video.duration/seq_len))
        if len(times) < seq_len:
            times.append(video.duration)
        times = times[:seq_len]

        # extract 2D points from csv
        data = np.genfromtxt(self.landmarks_path, delimiter=',')[1:]
        lm_times = [int(np.ceil(t)) for t in list(np.arange(0, len(data), len(data) / seq_len))]
        if len(lm_times) < seq_len:
            lm_times.append(len(data) - 1)
        lm_times = lm_times[:seq_len]
        index_x = (298, 366)
        index_y = (366, 433)
        landmarks_2d_x = [data[t, index_x[0] - 1:index_x[1] - 1] * (1 / 1280) for t in lm_times]
        landmarks_2d_y = [data[t, index_y[0] - 1:index_y[1]] * (1 / 720) for t in lm_times]

        for i, t in enumerate(times):
            img = cv2.cvtColor(video.get_frame(t), cv2.COLOR_BGR2RGB)
            # extract roi from landmarks and crop
            xs, ys = landmarks_2d_x[i], landmarks_2d_y[i]
            bottom = int(max(ys) * img.shape[0])
            right = int(max(xs) * img.shape[1])
            top = int(min(ys) * img.shape[0])
            left = int(min(xs) * img.shape[1])

            cropped = cv2.resize(img[top:bottom, left:right, :], target_resolution)
            cv2.imwrite(os.path.join(self.frames_folder, "frame_{0:.2f}.jpg".format(t)), cropped)

        print("Video duration {} seconds. Extracted {} frames".format(video.duration, len(times)))

In [ ]:
import numpy as np
import glob
import os
import argparse
from tqdm import tqdm
import librosa



def preprocess_video(dataset_folder):
    #make for each actor folder
    preprocessed_folder='/content/drive/MyDrive/graduation project/video/marwa/new_features/'
    landmark_folder='/content/drive/MyDrive/'
    output_folder = os.path.join(preprocessed_folder, "preprocessed_song")
    landmark_folder = os.path.join(landmark_folder, "landmarks")

    for each_actor in glob.glob(os.path.join(dataset_folder, "Actor*")):
        actor_name = os.path.basename(each_actor)
        output_actor_folder = os.path.join(output_folder, actor_name)
        print("output actor folder : "+output_actor_folder)
        if not os.path.exists(output_actor_folder):
            os.makedirs(output_actor_folder)

        for each_video in glob.glob(os.path.join(each_actor, "*.mp4")):
            print("each video :"+each_video)
            name = each_video.split("/")[-1].split("-")[0]
            print("name : "+name)
            if int(name) == 2: #video only
                continue
            video_name = os.path.basename(each_video)
            print('video name : '+video_name)
            landmark_path = os.path.join(landmark_folder, video_name[:-4] + '.csv')
            print('landmark path : '+landmark_path)
            frames_folder = os.path.join(output_actor_folder, video_name)
            print("frames folder : "+frames_folder)
            if not os.path.exists(frames_folder):
                os.mkdir(frames_folder)
            print("frame folder : "+frames_folder)    
            video_processor = VideoProcessor(video_path=each_video, landmark_path=landmark_path,
                                             output_folder=frames_folder, extract_audio=True)
            video_processor.preprocess(seq_len=30, target_resolution=(224, 224))

In [ ]:

print("Processing videos...")
preprocess_video('/content/drive/MyDrive/video_song/')

   

print("Preprocessed dataset located in ", os.path.join('/content/drive/MyDrive/graduation project/video/marwa/new_features/', 'preprocessed_song'))

Streaming output truncated to the last 5000 lines.
frames folder : /content/drive/MyDrive/graduation project/video/marwa/new_features/preprocessed_song/Actor_05/01-02-06-01-01-01-05.mp4
frame folder : /content/drive/MyDrive/graduation project/video/marwa/new_features/preprocessed_song/Actor_05/01-02-06-01-01-01-05.mp4
self video path : /content/drive/MyDrive/video_song/Actor_05/01-02-06-01-01-01-05.mp4
Video duration 4.67 seconds. Extracted 30 frames
each video :/content/drive/MyDrive/video_song/Actor_05/01-02-06-01-01-02-05.mp4
name : 01
video name : 01-02-06-01-01-02-05.mp4
landmark path : /content/drive/MyDrive/landmarks/01-02-06-01-01-02-05.csv
frames folder : /content/drive/MyDrive/graduation project/video/marwa/new_features/preprocessed_song/Actor_05/01-02-06-01-01-02-05.mp4
frame folder : /content/drive/MyDrive/graduation project/video/marwa/new_features/preprocessed_song/Actor_05/01-02-06-01-01-02-05.mp4
self video path : /content/drive/MyDrive/video_song/Actor_05/01-02-06-01-0

## check output folder

In [ ]:
dataset_folder='/content/drive/MyDrive/Video_Speech/'
preprocessed_folder='/content/drive/MyDrive/graduation project/video/marwa/new_features/'
landmark_folder='/content/drive/MyDrive/landmarks/'
output_folder = os.path.join(preprocessed_folder, "preprocessed_speech")
c=0
for each_actor in glob.glob(os.path.join(dataset_folder, "Actor*")):
        actor_name = os.path.basename(each_actor)
        output_actor_folder = os.path.join(output_folder, actor_name)
        print("output actor folder : "+output_actor_folder)
        c+=1

output actor folder : /content/drive/MyDrive/graduation project/video/marwa/new_features/preprocessed_speech/Actor_07
output actor folder : /content/drive/MyDrive/graduation project/video/marwa/new_features/preprocessed_speech/Actor_06
output actor folder : /content/drive/MyDrive/graduation project/video/marwa/new_features/preprocessed_speech/Actor_08
output actor folder : /content/drive/MyDrive/graduation project/video/marwa/new_features/preprocessed_speech/Actor_09
output actor folder : /content/drive/MyDrive/graduation project/video/marwa/new_features/preprocessed_speech/Actor_10
output actor folder : /content/drive/MyDrive/graduation project/video/marwa/new_features/preprocessed_speech/Actor_11
output actor folder : /content/drive/MyDrive/graduation project/video/marwa/new_features/preprocessed_speech/Actor_12
output actor folder : /content/drive/MyDrive/graduation project/video/marwa/new_features/preprocessed_speech/Actor_13
output actor folder : /content/drive/MyDrive/graduation 

In [ ]:
c

24

## create dataframe for frames

In [ ]:
# Create DataFrame for Data intel
folder_path ='/content/drive/MyDrive/graduation project/video/marwa/new_features/preprocessed_speech/'
dir_list = os.listdir(folder_path)
dir_list.sort()
print (dir_list)
data_df = pd.DataFrame(columns=['path', 'actor','file_name', 'emotion'])
count = 0
for i in dir_list:
    file_list = os.listdir(folder_path+i)
    print(file_list)
    for f in file_list:
        print(f)
        nm = f.split('.')[0].split('-')
        #print(nm)
        path = '/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/'+ i + '/' +f+ '/frames/' 
        #print(len(nm))
        actor = int(nm[6])
        emotion = int(nm[2])-1
        if(emotion==7 or emotion==6):
          continue
        data_df.loc[count] = [path, actor,f, emotion]
        count += 1

['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05', 'Actor_06', 'Actor_07', 'Actor_08', 'Actor_09', 'Actor_10', 'Actor_11', 'Actor_12', 'Actor_13', 'Actor_14', 'Actor_15', 'Actor_16', 'Actor_17', 'Actor_18', 'Actor_19', 'Actor_20', 'Actor_21', 'Actor_22', 'Actor_23', 'Actor_24']
['01-01-01-01-01-01-01.mp4', '01-01-02-01-01-01-01.mp4', '01-01-02-01-02-01-01.mp4', '01-01-02-01-02-02-01.mp4', '01-01-02-01-01-02-01.mp4', '01-01-01-01-02-01-01.mp4', '01-01-01-01-02-02-01.mp4', '01-01-01-01-01-02-01.mp4', '01-01-03-01-01-01-01.mp4', '01-01-02-02-01-01-01.mp4', '01-01-02-02-02-01-01.mp4', '01-01-02-02-02-02-01.mp4', '01-01-02-02-01-02-01.mp4', '01-01-03-02-02-01-01.mp4', '01-01-03-01-02-02-01.mp4', '01-01-03-02-01-01-01.mp4', '01-01-03-02-01-02-01.mp4', '01-01-03-01-02-01-01.mp4', '01-01-03-01-01-02-01.mp4', '01-01-04-01-02-02-01.mp4', '01-01-04-01-01-02-01.mp4', '01-01-04-02-01-01-01.mp4', '01-01-03-02-02-02-01.mp4', '01-01-04-01-01-01-01.mp4', '01-01-04-01-02-01-01.mp4', '01-01-04-

In [ ]:
# Create DataFrame for Data intel
folder_path ='/content/drive/MyDrive/graduation project/video/marwa/new_features/preprocessed_song/'
dir_list = os.listdir(folder_path)
dir_list.sort()
print (dir_list)
#data_df = pd.DataFrame(columns=['path', 'actor','file_name', 'emotion'])
count = 1056
for i in dir_list:
    file_list = os.listdir(folder_path+i)
    print(file_list)
    for f in file_list:
        print(f)
        nm = f.split('.')[0].split('-')
        #print(nm)
        path = '/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_song/'+ i + '/' +f+ '/frames/' 
        #print(len(nm))
        actor = int(nm[6])
        emotion = int(nm[2])-1
        if(emotion==7 or emotion==6):
          continue
        data_df.loc[count] = [path, actor,f, emotion]
        count += 1

['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05', 'Actor_06', 'Actor_07', 'Actor_08', 'Actor_09', 'Actor_10', 'Actor_11', 'Actor_12', 'Actor_13', 'Actor_14', 'Actor_15', 'Actor_16', 'Actor_17', 'Actor_19', 'Actor_20', 'Actor_21', 'Actor_22', 'Actor_23', 'Actor_24']
['01-02-01-01-01-02-01.mp4', '01-02-01-01-01-01-01.mp4', '01-02-01-01-02-01-01.mp4', '01-02-02-01-01-01-01.mp4', '01-02-01-01-02-02-01.mp4', '01-02-02-01-01-02-01.mp4', '01-02-02-01-02-02-01.mp4', '01-02-02-02-01-01-01.mp4', '01-02-02-01-02-01-01.mp4', '01-02-02-02-02-02-01.mp4', '01-02-02-02-01-02-01.mp4', '01-02-02-02-02-01-01.mp4', '01-02-03-01-02-01-01.mp4', '01-02-03-01-01-02-01.mp4', '01-02-03-01-01-01-01.mp4', '01-02-03-02-01-01-01.mp4', '01-02-03-01-02-02-01.mp4', '01-02-03-02-01-02-01.mp4', '01-02-03-02-02-02-01.mp4', '01-02-03-02-02-01-01.mp4', '01-02-04-01-01-01-01.mp4', '01-02-04-01-02-01-01.mp4', '01-02-04-01-01-02-01.mp4', '01-02-04-01-02-02-01.mp4', '01-02-04-02-01-02-01.mp4', '01-02-04-02-01-01-01.

In [ ]:
count

2068

In [ ]:
data_df

,path,actor,file_name,emotion
0,/content/drive/MyDrive/graduation project/vide...,1,01-01-01-01-01-01-01.mp4,0
1,/content/drive/MyDrive/graduation project/vide...,1,01-01-02-01-01-01-01.mp4,1
2,/content/drive/MyDrive/graduation project/vide...,1,01-01-02-01-02-01-01.mp4,1
3,/content/drive/MyDrive/graduation project/vide...,1,01-01-02-01-02-02-01.mp4,1
4,/content/drive/MyDrive/graduation project/vide...,1,01-01-02-01-01-02-01.mp4,1
...,...,...,...,...
2063,/content/drive/MyDrive/graduation project/vide...,24,01-02-06-02-01-02-24.mp4,5
2064,/content/drive/MyDrive/graduation project/vide...,24,01-02-06-02-01-01-24.mp4,5
2065,/content/drive/MyDrive/graduation project/vide...,24,01-02-06-01-02-02-24.mp4,5
2066,/content/drive/MyDrive/graduation project/vide...,24,01-02-06-02-02-01-24.mp4,5


In [ ]:
data_df['path']

0       /content/drive/MyDrive/graduation project/vide...
1       /content/drive/MyDrive/graduation project/vide...
2       /content/drive/MyDrive/graduation project/vide...
3       /content/drive/MyDrive/graduation project/vide...
4       /content/drive/MyDrive/graduation project/vide...
                              ...                        
1051    /content/drive/MyDrive/graduation project/vide...
1052    /content/drive/MyDrive/graduation project/vide...
1053    /content/drive/MyDrive/graduation project/vide...
1054    /content/drive/MyDrive/graduation project/vide...
1055    /content/drive/MyDrive/graduation project/vide...
Name: path, Length: 1056, dtype: object

In [ ]:
# Saving joblib files to not load them again with the loop above

import joblib

X_name = 'RAVDESS_video.joblib'
save_dir = '/content/drive/MyDrive/graduation project/video/marwa/new_features/'

savedX = joblib.dump(data_df, os.path.join(save_dir, X_name))


In [ ]:
#load saved data
import joblib
data = joblib.load('/content/drive/MyDrive/graduation project/video/marwa/new_features/RAVDESS_video.joblib')

In [ ]:
data

,path,actor,file_name,emotion
0,/content/drive/MyDrive/graduation project/vide...,1,01-01-01-01-01-01-01.mp4,0
1,/content/drive/MyDrive/graduation project/vide...,1,01-01-02-01-01-01-01.mp4,1
2,/content/drive/MyDrive/graduation project/vide...,1,01-01-02-01-02-01-01.mp4,1
3,/content/drive/MyDrive/graduation project/vide...,1,01-01-02-01-02-02-01.mp4,1
4,/content/drive/MyDrive/graduation project/vide...,1,01-01-02-01-01-02-01.mp4,1
...,...,...,...,...
2063,/content/drive/MyDrive/graduation project/vide...,24,01-02-06-02-01-02-24.mp4,5
2064,/content/drive/MyDrive/graduation project/vide...,24,01-02-06-02-01-01-24.mp4,5
2065,/content/drive/MyDrive/graduation project/vide...,24,01-02-06-01-02-02-24.mp4,5
2066,/content/drive/MyDrive/graduation project/vide...,24,01-02-06-02-02-01-24.mp4,5


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1056 entries, 0 to 1055
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   path       1056 non-null   object
 1   actor      1056 non-null   object
 2   file_name  1056 non-null   object
 3   emotion    1056 non-null   object
dtypes: object(4)
memory usage: 41.2+ KB


In [ ]:
data.actor.unique()

array([1, 2], dtype=object)

In [ ]:
data.emotion.unique()

array([0, 1, 2, 3, 4, 5], dtype=object)

In [ ]:
data.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
1051    False
1052    False
1053    False
1054    False
1055    False
Length: 1056, dtype: bool

In [ ]:
print(20 in data['actor'].unique())

False


In [ ]:
x_test=[]
y_test=[]
for index, row in test.iterrows():
  pathh=row['path']
  for filename in glob.glob(pathh+'*.jpg'):
    if filename is not None:
      print(filename)
      y_test.append(row['emotion'])
      x_test.append(cv2.imread(os.path.join(filename),0))

/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_14/01-01-03-01-01-01-14.mp4/frames/frame_0.00.jpg
/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_14/01-01-03-01-01-01-14.mp4/frames/frame_0.12.jpg
/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_14/01-01-03-01-01-01-14.mp4/frames/frame_0.25.jpg
/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_14/01-01-03-01-01-01-14.mp4/frames/frame_0.38.jpg
/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_14/01-01-03-01-01-01-14.mp4/frames/frame_0.50.jpg
/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_14/01-01-03-01-01-01-14.mp4/frames/frame_0.62.jpg
/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_14/01-01-03-01-01-01-14.mp4/frames/frame_0.75.jpg
/content/drive/MyDrive/grad

In [ ]:
x_valid=[]
y_valid=[]
for index, row in valid.iterrows():
  pathh=row['path']
  for filename in glob.glob(pathh+'*.jpg'):
    if filename is not None:
      print(filename)
      y_valid.append(row['emotion'])
      x_valid.append(cv2.imread(os.path.join(filename),0))

/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_01/01-01-03-02-01-02-01.mp4/frames/frame_0.00.jpg
/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_01/01-01-03-02-01-02-01.mp4/frames/frame_0.12.jpg
/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_01/01-01-03-02-01-02-01.mp4/frames/frame_0.25.jpg
/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_01/01-01-03-02-01-02-01.mp4/frames/frame_0.37.jpg
/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_01/01-01-03-02-01-02-01.mp4/frames/frame_0.49.jpg
/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_01/01-01-03-02-01-02-01.mp4/frames/frame_0.61.jpg
/content/drive/MyDrive/graduation project/video/marwa/features/preprocessed_speech/Actor_01/01-01-03-02-01-02-01.mp4/frames/frame_0.74.jpg
/content/drive/MyDrive/grad

KeyboardInterrupt: ignored